In [ ]:

import warnings

import matplotlib.pyplot as plt
# import MultiTrade
import numpy as np
import pandas as pd
from IPython.display import display
from neuralforecast.core import NeuralForecast

from neuralforecast.auto import AutoBiTCN, AutoNBEATS, AutoTFT
from neuralforecast.losses.pytorch import RMSE

# from statsforecast import StatsForecast

from utils.utils import build_market_image,train_test_split_data
from configs import futures_defaults as defaults
import datetime
warnings.filterwarnings("ignore")

In [ ]:
COIN_PAIRS=defaults.COIN_PAIRS
target_pair=defaults.target_pair
time_frame=defaults.time_frame
DATA_DIR=defaults.DATA_DIR

In [3]:

horizon = 7
context_length=5*horizon
split_n_days=30
round_time='H'
id_col='symbol'
save_path='Futures_forecasting_model'

In [ ]:

data=build_market_image(target_pair=target_pair,time_frame=time_frame,axis=0,data_dir=DATA_DIR)
# data.index
train_data,test_data=train_test_split_data(data=data,n_days=30)
# data['symbol'].unique()

# test_data
data

['data_futures/coinbase-ETHUSD_Futures-1h.pkl', 'data_futures/coinbase-BTCUSD_Futures-1h.pkl', 'data_futures/coinbase-LTCUSD_Futures-1h.pkl', 'data_futures/coinbase-SUIUSD_Futures-1h.pkl', 'data_futures/coinbase-DOGEUSD_Futures-1h.pkl']


0it [00:00, ?it/s]

100%|██████████| 78/78 [00:00<00:00, 14716.86it/s]
5it [00:00, 10.00it/s]


2024-10-16 11:00:00 2024-11-15 11:00:00


,low,high,open,close,volume,symbol,symbol_name,y,feature_hour_of_day,feature_day_of_week,...,feature_day_of_year,feature_BBANDS_BB_UPPER,feature_BBANDS_BB_MIDDLE,feature_BBANDS_BB_LOWER,feature_MACD_MACD,feature_MACD_SIGNAL,feature_14_period_RSI,feature_14_period_STOCH_%K,ds,unique_id
date_close,,,,,,,,,,,,,,,,,,,,,
2024-05-15 14:00:00,2945.55000,2977.10000,2951.05000,2976.55000,4.369228e+03,ETHUSD_Futures,ETHUSD_Futures,2976.55000,0.108696,-0.166667,...,-0.130137,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2024-05-15 14:00:32,31
2024-05-15 13:00:00,2936.05000,2983.91000,2947.40000,2951.49000,8.402044e+03,ETHUSD_Futures,ETHUSD_Futures,2951.49000,0.065217,-0.166667,...,-0.130137,0.000000,0.000000,0.000000,-0.562244,-0.312358,0.000000,0.000000,2024-05-15 13:00:48,31
2024-05-15 12:00:00,2893.04000,2958.56000,2898.97000,2947.27000,8.467402e+03,ETHUSD_Futures,ETHUSD_Futures,2947.27000,0.021739,-0.166667,...,-0.130137,0.000000,0.000000,0.000000,-0.847272,-0.531585,0.000000,0.000000,2024-05-15 12:01:04,31
2024-05-15 11:00:00,2887.12000,2909.61000,2908.23000,2898.87000,3.628626e+03,ETHUSD_Futures,ETHUSD_Futures,2898.87000,-0.065217,-0.166667,...,-0.130137,0.000000,0.000000,0.000000,-2.703607,-1.267365,0.000000,0.000000,2024-05-15 10:59:12,31
2024-05-15 10:00:00,2906.83000,2924.25000,2920.96000,2908.68000,3.512863e+03,ETHUSD_Futures,ETHUSD_Futures,2908.68000,-0.108696,-0.166667,...,-0.130137,0.000000,0.000000,0.000000,-3.265032,-1.861626,12.503829,0.000000,2024-05-15 09:59:28,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-06 19:00:00,0.19247,0.19591,0.19464,0.19525,3.157381e+07,DOGEUSD_Futures,DOGEUSD_Futures,0.19525,0.282609,-0.166667,...,0.349315,0.201448,0.194178,0.186908,0.000505,0.000148,49.212263,40.163934,2024-11-06 18:59:12,29
2024-11-06 18:00:00,0.19220,0.19620,0.19239,0.19466,2.582273e+07,DOGEUSD_Futures,DOGEUSD_Futures,0.19466,0.239130,-0.166667,...,0.349315,0.201357,0.194440,0.187523,0.000356,0.000189,47.934794,36.443884,2024-11-06 17:59:28,29
2024-11-06 17:00:00,0.18948,0.19404,0.19399,0.19230,2.862901e+07,DOGEUSD_Futures,DOGEUSD_Futures,0.19230,0.195652,-0.166667,...,0.349315,0.200700,0.194722,0.188744,0.000046,0.000161,43.113785,21.563682,2024-11-06 16:59:44,29


In [5]:
front=['y','ds','symbol']
exo_gen_cols=data.filter(like='feature_').columns.tolist()
cols=front+[c for c in exo_gen_cols if c not in front]

train_data=train_data[cols]
test_data=test_data[cols]
# train_data
# data[id_col].unique()
# 

In [6]:
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [7]:
forecast_horizon=6
backend='optuna'
BiTCN_config=AutoBiTCN.get_default_config(h=forecast_horizon,backend=backend)

TFT_config=AutoTFT.get_default_config(h=forecast_horizon,backend=backend)
NBEATS_config=AutoNBEATS.get_default_config(h=forecast_horizon,backend=backend)

In [8]:
BiTCN_MODEL= AutoBiTCN(h=forecast_horizon,
                  loss=RMSE(),
                  config=BiTCN_config,
                  search_alg=optuna.samplers.TPESampler(),
                  backend='optuna',
                  num_samples=10)

TFT_MODEL= AutoTFT(h=forecast_horizon,
                  loss=RMSE(),
                  config=TFT_config,
                  search_alg=optuna.samplers.TPESampler(),
                  backend='optuna',
                  num_samples=10)

NBEATS_MODEL= AutoNBEATS(h=forecast_horizon,
                  loss=RMSE(),
                  config=NBEATS_config,
                  search_alg=optuna.samplers.TPESampler(),
                  backend='optuna',
                  num_samples=10)

models=[BiTCN_MODEL,TFT_MODEL,NBEATS_MODEL]

In [9]:

model = NeuralForecast(models=models,
                    local_scaler_type='robust',
                    
                     freq='1h')

In [10]:
model.fit(train_data,
        val_size=horizon,
        time_col='ds',                    
        target_col='y',
        id_col=id_col,
        
        )


Seed set to 16
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name          | Type          | Params | Mode 
--------------------------------------------------------
0 | loss          | RMSE          | 0      | train
1 | padder_train  | ConstantPad1d | 0      | train
2 | scaler        | TemporalNorm  | 0      | train
3 | lin_hist      | Linear        | 32     | train
4 | drop_hist     | Dropout       | 0      | train
5 | net_bwd       | Sequential    | 5.4 K  | train
6 | drop_temporal | Dropout       | 0      | train
7 | temporal_lin1 | Linear        | 304    | train
8 | temporal_lin2 | Linear        | 102    | train
9 | output_lin    | Linear        | 17     | train
--------------------------------------------------------
5.8 K     Trainable params
0         Non-trainable params
5.8 K     Total params
0.023     Total estimated model params size (MB)
30        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=1000` reached.


In [11]:
model.save(save_path,overwrite=True)

In [12]:
test_dates=test_data.sort_values('ds')['ds'].unique()
last_date=train_data['ds'].unique()[-1]

In [13]:
test_data

,y,ds,symbol,feature_hour_of_day,feature_day_of_week,feature_day_of_month,feature_day_of_year,feature_BBANDS_BB_UPPER,feature_BBANDS_BB_MIDDLE,feature_BBANDS_BB_LOWER,feature_MACD_MACD,feature_MACD_SIGNAL,feature_14_period_RSI,feature_14_period_STOCH_%K
0,67898.5000,2024-10-16 10:59:12,BTCUSD_Futures,-0.065217,-0.166667,0.000000,0.291781,68096.232412,67640.465000,67184.697588,75.707337,59.415020,55.712167,59.835260
1,67737.0000,2024-10-16 09:59:28,BTCUSD_Futures,-0.108696,-0.166667,0.000000,0.291781,68063.700405,67669.890000,67276.079595,72.314078,61.994832,52.104654,47.630923
2,67409.7000,2024-10-16 08:59:44,BTCUSD_Futures,-0.152174,-0.166667,0.000000,0.291781,68053.775123,67674.165000,67294.554877,42.722047,58.140275,45.652814,25.296530
3,67117.2000,2024-10-16 08:00:00,BTCUSD_Futures,-0.152174,-0.166667,0.000000,0.291781,68078.596670,67668.380000,67258.163330,-4.282787,45.655662,40.791623,14.027505
4,66965.4000,2024-10-16 07:00:16,BTCUSD_Futures,-0.195652,-0.166667,0.000000,0.291781,68153.858682,67637.265000,67120.671318,-53.170531,25.890424,38.500383,11.962010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2730,2.2985,2024-11-06 18:59:12,SUIUSD_Futures,0.282609,-0.166667,-0.333333,0.349315,2.343484,2.280995,2.218506,0.000453,-0.003431,49.901168,48.202247
2731,2.2724,2024-11-06 17:59:28,SUIUSD_Futures,0.239130,-0.166667,-0.333333,0.349315,2.343395,2.280830,2.218265,-0.001701,-0.003085,45.482628,33.539326
2732,2.2526,2024-11-06 16:59:44,SUIUSD_Futures,0.195652,-0.166667,-0.333333,0.349315,2.343058,2.281130,2.219202,-0.004948,-0.003458,42.414379,22.415730
2733,2.2690,2024-11-06 16:00:00,SUIUSD_Futures,0.195652,-0.166667,-0.333333,0.349315,2.343024,2.281000,2.218976,-0.006128,-0.003992,45.682855,19.775873


In [14]:
fcst_df = model.predict()
plot_df=train_data.groupby('symbol').tail(context_length)
plot_df
# fcst_df.tail()

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

,y,ds,symbol,feature_hour_of_day,feature_day_of_week,feature_day_of_month,feature_day_of_year,feature_BBANDS_BB_UPPER,feature_BBANDS_BB_MIDDLE,feature_BBANDS_BB_LOWER,feature_MACD_MACD,feature_MACD_SIGNAL,feature_14_period_RSI,feature_14_period_STOCH_%K
4073,67079.7000,2024-10-17 21:00:48,BTCUSD_Futures,0.413043,0.000000,0.033333,0.294521,68788.552674,67886.250000,66983.947326,-251.085570,-176.169109,35.200410,10.279144
4074,66931.3000,2024-10-17 20:01:04,BTCUSD_Futures,0.369565,0.000000,0.033333,0.294521,68683.945341,67792.865000,66901.784659,-289.031227,-198.741533,33.416925,7.449195
4075,66830.8000,2024-10-17 18:59:12,BTCUSD_Futures,0.282609,0.000000,0.033333,0.294521,68570.555952,67697.475000,66824.394048,-323.484021,-223.690030,32.226105,3.758180
4076,66930.9000,2024-10-17 17:59:28,BTCUSD_Futures,0.239130,0.000000,0.033333,0.294521,68489.355406,67622.760000,66756.164594,-338.805338,-246.713092,34.721297,13.759331
4077,67277.5000,2024-10-17 16:59:44,BTCUSD_Futures,0.195652,0.000000,0.033333,0.294521,68413.645647,67577.160000,66740.674353,-319.299187,-261.230311,42.601241,34.458047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20535,1.9943,2024-10-16 15:00:16,SUIUSD_Futures,0.152174,-0.166667,0.000000,0.291781,2.182716,2.091790,2.000864,-0.012192,0.002536,33.677595,8.297414
20536,2.0352,2024-10-16 14:00:32,SUIUSD_Futures,0.108696,-0.166667,0.000000,0.291781,2.183797,2.090835,1.997873,-0.013341,-0.000639,43.265164,30.334052
20537,2.0246,2024-10-16 13:00:48,SUIUSD_Futures,0.065217,-0.166667,0.000000,0.291781,2.184744,2.087250,1.989756,-0.014935,-0.003498,41.587228,25.602241
20538,2.0388,2024-10-16 12:01:04,SUIUSD_Futures,0.021739,-0.166667,0.000000,0.291781,2.183932,2.084260,1.984588,-0.014881,-0.005775,44.682284,33.557423


In [15]:
from statsforecast import StatsForecast


ModuleNotFoundError: No module named 'statsforecast'

In [ ]:
StatsForecast.plot(plot_df, fcst_df, engine='matplotlib',id_col=id_col, max_insample_length=context_length,)


In [ ]:
pred_frame=test_data.copy()

In [ ]:
model=NeuralForecast.load(save_path)

In [ ]:
fcst_df = model.predict(pred_frame)

# fcst_df.head()

In [ ]:
StatsForecast.plot(pred_frame, fcst_df, engine='matplotlib',id_col=id_col, max_insample_length=context_length,)


In [ ]:
test_data

In [ ]:
# test_data=test_data.drop('close_date',axis=1)
test_data[exo_gen_cols].values.dtype

In [ ]:
model.dataset, model.uids, model.last_dates, model.ds = model._prepare_fit(
                df=test_data,
                static_df=None,
                sort_df=model.sort_df,
                predict_only=False,
                id_col=model.id_col,
                time_col=model.time_col,
                target_col=model.target_col,
            )


In [ ]:
test_pred_insample = model.predict_insample(step_size=horizon)


In [ ]:
test_pred_insample['mean']=test_pred_insample.filter(like='Auto').mean(axis=1)
# test_pred_insample['mean']
# test_pred_insample['mean']
# test_pred_insample.filter(like='Auto').mean(axis=1)
# test_pred_insample.filter(like='Auto').mean(axis=1)

In [ ]:
# test_pred_insample.columns=test_pred_insample.columns.str.replace('Auto','')

In [ ]:
test_data['close']=test_data['y'].copy()


In [ ]:
idx,cut_data=next(iter(test_pred_insample.groupby(['cutoff','symbol'])))
def flatten_preds(idx,cut_data):
    t_off_pred,symb=idx
    pred_cols=cut_data.filter(like='Auto').columns
    cut_data['mean_pred']=cut_data[pred_cols].mean(axis=1)
    cut_data=cut_data.drop(pred_cols,axis=1)
    t_cut=cut_data.T

    t_cut=t_cut.drop('cutoff')
    t_cut=t_cut.drop('ds')
    t_cut=t_cut.drop('y')
    # print(t_cut)
    h_cols=[f'H{i}' for i in range(horizon-1)]
    # print (h_cols)
    t_cut.columns=h_cols


    flat_cols=[f'feature_{model}_{horizon}' for model in t_cut.index for horizon in t_cut.columns]
    pred_values=t_cut.values.flatten()

    flat_df=pd.DataFrame([pred_values],columns=flat_cols)
    id_cols=['ds','symbol']

    flat_df[id_cols]=t_off_pred,symb
    flat_df=  flat_df[id_cols+flat_cols]
    return flat_df

def simulate_forecasts(model,df):
    model.dataset, model.uids, model.last_dates, model.ds = model._prepare_fit(
                df=test_data,
                static_df=None,
                sort_df=model.sort_df,
                predict_only=False,
                id_col=model.id_col,
                time_col=model.time_col,
                target_col=model.target_col,
            )
    
    
    return model.predict_insample(step_size=1)

def prepare_forecast_data(model,test_data):
    
    pred_df= simulate_forecasts(model,test_data)
    flattened_preds=pd.concat([flatten_preds(idx,cut_data) for idx,cut_data in pred_df.groupby(['cutoff','symbol'])])
    return flattened_preds



prepare_forecast_data(model,test_data)

In [ ]:
cut_data
for symb,cut in test_pred_insample.groupby('symbol'):
    cut['mean']=cut.filter(like='Auto').mean(axis=1)

    plt.figure(figsize=(10, 5))
    
    plt.plot(cut['ds'], cut['y'], label='True')
    plt.plot(cut['ds'], cut['mean'] ,label='mean Forecast')
    for model in cut.filter(like='Auto').columns:
        plt.plot(cut['ds'], cut[model], label=f'{model} Forecast')
    # plt.plot(cut['ds'], cut['NBEATS'], label='NBEATS Forecast')
    # plt.plot(cut['ds'], cut['BiTCN'], label='BiTCN Forecast')
    # plt.plot(cut['ds'], cut['TFT'], label='TFT Forecast')
    # plt.axvline(cut['ds'].iloc[-12], color='black', linestyle='--', label='Train-Test Split')
    plt.xlabel('Timestamp [t]')
    plt.ylabel(f'{symb} Price')
    plt.grid()
    plt.legend()

In [ ]:
# StatsForecast.plot(test_pred_insample, engine='matplotlib',time_col='ds',id_col=id_col, max_insample_length=horizon,)
StatsForecast.plot(test_pred_insample, engine='matplotlib',time_col='ds',id_col=id_col,target_col='AutoNBEATS', max_insample_length=horizon,)
